**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [カラーのオブジェクトの生成](#toc5_)    
- [関数の定義](#toc6_)    
- [カメラの設定](#toc7_)    
- [アームの初期姿勢の設定](#toc8_)    
- [カメラ座標系におけるカラーのオブジェクトの位置を推定](#toc9_)    
- [ワールド座標系におけるカラーのオブジェクトの位置を推定](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは6軸のロボットアームを生成し、「アームの手先のカメラ」から、指定した色の物体の位置を推定する方法を解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 3.5
cameraYaw = 180.0 # deg
cameraPitch = -40 # deg
cameraTargetPosition = [0, 0.5, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)

In [3]:
# ロボットの読み込み
armStartPos = [0, 0, 0.0]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple6d_arm_with_gripper.urdf", armStartPos, armStartOrientation, useFixedBase=True)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[カラーのオブジェクトの生成](#toc0_)

カメラで検出したいカラーのオブジェクトを生成します。

In [4]:
# ここを変えると結果が変わります（手先のカメラの画角に収まるように設定してください）####
colorBoxPos = [-0.5, 0.2, 0.05] # 色物体の初期位置(x, y, z)を設定
#################################################################################

colorBoxId = pybullet.loadURDF("../urdf/simple_box.urdf", colorBoxPos, pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0]), globalScaling=0.1, useFixedBase=True)

# <a id='toc6_'></a>[関数の定義](#toc0_)

In [6]:
import cv2
import numpy as np

def DetectColorObjPose(targetRGB, rgbImg, depthImg):
    """
    最初に検出された色物体の中心位置、深度、姿勢を取得する関数
    
    Parameters
    ----------
    targetRGB : list
        検出したい色のRGB
    rgbImg : numpy.ndarray
        カメラ画像（RGB）
    depthImg : numpy.ndarray
        カメラ画像（Depth）

    Returns
    -------
    obj_pose : numpy.ndarray
        色物体の位置と姿勢（x, y, z, roll, pitch, yaw）
    """

    # カメラ画像をHSV形式に変換
    hsvImg = cv2.cvtColor(rgbImg, cv2.COLOR_RGB2HSV)

    # RGBをHSVに変換
    targetHsv = cv2.cvtColor(np.uint8([[targetRGB]]), cv2.COLOR_RGB2HSV)[0][0]

    # 検出したい色の範囲を指定
    lower = np.array([targetHsv[0]-10, 50, 50])
    upper = np.array([targetHsv[0]+10, 255, 255])

    # 指定した色のみを抽出
    mask = cv2.inRange(hsvImg, lower, upper)

    # 輪郭を抽出
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 最大面積の輪郭を取得
    max_area = 0
    max_area_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_contour = contour

    # 輪郭が見つからなかった場合
    if max_area_contour is None:
        return None
    
    # 輪郭の中心位置を取得
    M = cv2.moments(max_area_contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    # 輪郭の中心位置における深度を取得
    depth = depthImg[cy, cx]

    pos = [cx, cy, depth]
    
    return pos



def Rx(theta):
    """
    x軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸周りの回転行列
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    y軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸周りの回転行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    z軸周りの回転行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸周りの回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def Hx(theta):
    """
    x軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        x軸回りの同次変換行列
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    y軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        y軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    z軸回りの同次変換行列を求める

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        z軸回りの同次変換行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    平行移動の同次変換行列を求める

    Parameters
    ----------
    x : float
        x方向の移動量
    y : float
        y方向の移動量
    z : float
        z方向の移動量

    Returns
    -------
    numpy.ndarray
        平行移動の同次変換行列
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])

# <a id='toc7_'></a>[カメラの設定](#toc0_)
カメラに関する設定（焦点距離、内部パラメータなど）を定義します。

In [7]:
# カメラ設定
fov = 60 # Pybulletでは、垂直方向のfovを指定
imageWidth = 224 # 画像の幅
imageHeight = 224 # 画像の高さ
aspect = imageWidth / imageHeight # アスペクト比
near = 0.05 # カメラの最小距離
far = 5 # カメラの最大距離
projectionMatrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far) # 射影行列を計算

# 焦点距離を求める
fovRad = np.deg2rad(fov)
f = (imageHeight / 2) / np.tan(fovRad / 2)

# カメラの内部パラメータ
cameraMatrix = np.array([[f, 0, imageWidth//2],
                         [0, f, imageHeight//2],
                         [0, 0, 1]], dtype=np.float32)

# 歪み係数（ここでは、歪みがないと仮定）
distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc8_'></a>[アームの初期姿勢の設定](#toc0_)

In [8]:
# ロボットの各関節のインデックス
link1JointIdx = 0
link2JointIdx = 1
link3JointIdx = 2
link4JointIdx = 3
link5JointIdx = 4
link6JointIdx = 5
cameraIdx = 6
cameraTargetIdx = 7

# 各リンクの長さ
worldToArmOriginX = 0.0 # ワールド座標系から見たアームの原点のx座標
worldToArmOriginY = 0.0 # ワールド座標系から見たアームの原点のy座標
worldToArmOriginZ = 0.8 # ワールド座標系から見たアームの原点のz座標(「simple6d_arm_with_gripper.urdf」の base_link の z方向の長さ)
link1Length = 0.3 # link1の長さ(「simple6d_arm_with_gripper.urdf」の link1 の z方向の長さ)
link2Length = 0.5 # link2の長さ(「simple6d_arm_with_gripper.urdf」の link2 の z方向の長さ)
link3Length = 0.5 # link3の長さ(「simple6d_arm_with_gripper.urdf」の link3 の z方向の長さ)
link4Length = 0.1 # link4の長さ(「simple6d_arm_with_gripper.urdf」の link4 の z方向の長さ)
link5Length = 0.1 # link5の長さ(「simple6d_arm_with_gripper.urdf」の link5 の z方向の長さ)
link6Length = 0.15 # link6の長さ(「simple6d_arm_with_gripper.urdf」の link6 の z方向の長さ)
cameraZSize = 0.01 # カメラのz方向の長さ
cameraXOffset = 0.08 # カメラのリンク6座標系の原点からのx方向のオフセット

# ここを変えると結果が変わります（手先のカメラの画角にcolorBoxが収まるように設定してください）####
# 各関節の角度
joint1Angle = -15.0
joint2Angle = -80.0
joint3Angle = -100.0
joint4Angle = 0.0
joint5Angle = 0.0
joint6Angle = 10.0
########################################################################################

# 各関節の角度をラジアンに変換
q1 = np.deg2rad(joint1Angle)
q2 = np.deg2rad(joint2Angle)
q3 = np.deg2rad(joint3Angle)
q4 = np.deg2rad(joint4Angle)
q5 = np.deg2rad(joint5Angle)
q6 = np.deg2rad(joint6Angle)

# 各関節の角度を設定
pybullet.resetJointState(armId, link1JointIdx, q1)
pybullet.resetJointState(armId, link2JointIdx, q2)
pybullet.resetJointState(armId, link3JointIdx, q3)
pybullet.resetJointState(armId, link4JointIdx, q4)
pybullet.resetJointState(armId, link5JointIdx, q5)
pybullet.resetJointState(armId, link6JointIdx, q6)

# <a id='toc9_'></a>[カメラ座標系におけるカラーのオブジェクトの位置を推定](#toc0_)

In [9]:
# カメラの位置を取得
cameraLinkPose = pybullet.getLinkState(armId, cameraIdx)[0] # 手先のカメラリンクの位置
cameraTargetLinkPose = pybullet.getLinkState(armId, cameraTargetIdx)[0] # カメラリンクの少しだけ前に設定した仮想的なリンクの位置
cameraLinkOrientation = pybullet.getEulerFromQuaternion(pybullet.getLinkState(armId, cameraIdx)[1]) # 手先のカメラリンクの姿勢

# ロボットアームの姿勢に合わせて、カメラの上方向のベクトルを設定
RW1 = Rz(theta=0)@Ry(theta=0)@Rx(theta=0) # ワールド座標系 -> link1座標系の回転行列
R12 = Rz(theta=q1) # link1座標系 -> link2座標系
R23 = Ry(theta=q2) # link2座標系 -> link3座標系
R34 = Ry(theta=q3) # link3座標系 -> link4座標系
R45 = Rz(theta=q4) # link4座標系 -> link5座標系
R56 = Ry(theta=q5) # link5座標系 -> link6座標系
R6C = Rz(theta=q6 + np.deg2rad(-90.0)) # link6座標系 -> カメラ座標系（カメラを-90度回転すると、取得される画像が正しい向きになる）
R = RW1@ R12@R23@R34@R45@R56@R6C # link1座標系 -> カメラ座標系への回転行列
cameraUpVector = np.array([0, -1, 0]) # デフォルトのカメラの上方向のベクトル
rotateCameraUpVector = R@cameraUpVector # ロボットアームの姿勢に合わせたカメラの上方向のベクトル

# カメラのビュー行列を計算
viewMatrix = pybullet.computeViewMatrix(cameraEyePosition=[cameraLinkPose[0], cameraLinkPose[1], cameraLinkPose[2]],cameraTargetPosition=[cameraTargetLinkPose[0], cameraTargetLinkPose[1], cameraTargetLinkPose[2]],cameraUpVector=[rotateCameraUpVector[0], rotateCameraUpVector[1], rotateCameraUpVector[2]])

# カメラ画像を取得
_, _, rgbImg, depthImg, _ = pybullet.getCameraImage(
    width=imageWidth,
    height=imageHeight,
    viewMatrix=viewMatrix,
    projectionMatrix=projectionMatrix,
    renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
)

# 指定した色の物体の位置を取得
detectColorRGB = [0, 0, 255] # 検出したい色を指定（ここでは青色）
pos = DetectColorObjPose(detectColorRGB, rgbImg, depthImg)
pixelX = pos[0] # x座標（画像上の位置）
pixelY = pos[1] # y座標（画像上の位置）
normalZ = pos[2] # z座標（0～1に正規化された深度）
z = far * near / (far - (far - near) * normalZ) # 0～1に正規化された深度を距離[m]に変換
x = (pixelX - imageWidth//2) * z / f # pixelXを距離[m]に変換
y = (pixelY - imageHeight//2) * z / f # pixelYを距離[m]に変換

# <a id='toc10_'></a>[ワールド座標系におけるカラーのオブジェクトの位置を推定](#toc0_)

In [10]:
# ワールド座標系から見たカメラの位置を求める
TW1 = Hz(theta=0)@Hy(theta=0)@Hx(theta=0)@Hp(x=worldToArmOriginX, y=worldToArmOriginY, z=worldToArmOriginZ) # ワールド座標系 -> link1座標系の同次変換行列
T12 = Hz(theta=q1)@Hp(x=0, y=0, z=link1Length) # link1座標系 -> link2座標系
T23 = Hy(theta=q2)@Hp(x=0, y=0, z=link2Length) # link2座標系 -> link3座標系
T34 = Hy(theta=q3)@Hp(x=0, y=0, z=link3Length) # link3座標系 -> link4座標系
T45 = Hz(theta=q4)@Hp(x=0, y=0, z=link4Length) # link4座標系 -> link5座標系
T56 = Hy(theta=q5)@Hp(x=0, y=0, z=link5Length) # link5座標系 -> link6座標系
T6C = Hz(theta=q6)@Hp(x=cameraXOffset, y=0, z=link6Length-cameraZSize/2) # link6座標系 -> カメラ座標系
originPos = np.array([0, 0, 0, 1]) # 原点位置
cameraPosWorld = TW1@T12@T23@T34@T45@T56@T6C@originPos # ワールド座標系から見たカメラ座標系の原点位置
cameraPosWorld = cameraPosWorld[:3] # 同次変換行列によって求めた座標は[x, y, z, 1]の形式なので、[x, y, z]の形式に変換
colorObjPosCamera = np.array([x, y, z]) # カメラ座標系から見た「カラーのオブジェクト」の中心座標
colorObjPosWorld = cameraPosWorld + R@colorObjPosCamera # ワールド座標系から見た「カラーオブジェクト」の位置

# Pybuletの画面上に、ワールド座標系から見た「手先カメラ位置」→「推定したカラーのオブジェクトの位置」に向かう赤色の線を描画
pybullet.addUserDebugLine(cameraPosWorld, colorObjPosWorld, [1, 0, 0], lineWidth=2)

# ボックスの「真の位置」と「推定した位置」を表示
boxPos, _ = pybullet.getBasePositionAndOrientation(colorBoxId)
pybullet.addUserDebugText(f"true box pose ({boxPos[0]:.3f}, {boxPos[1]:.3f}, {boxPos[2]:.3f})", [2.0, 0.5, 0], textColorRGB=[1, 0, 0], textSize=1.3)
pybullet.addUserDebugText(f"eye in hand, estimate box pose ({colorObjPosWorld[0]:.3f}, {colorObjPosWorld[1]:.3f}, {colorObjPosWorld[2]:.3f})", [2.0, 1.0, 0.0], textColorRGB=[1, 0, 0], textSize=1.3)

color box obj pose:  -0.5 0.2 0.05
estimate color obj pose -0.4979110352358044 0.201106116292795 0.10000000370971596


2